In [ ]:
# É necessário que tenha o pandas e o prettytable instalados, caso não tenha realizar o pip abaixo:
# pip install prettytable
# pip install pandas

In [ ]:
# Importando os pacotes
import pandas as pd
from prettytable import PrettyTable
from itertools import zip_longest
import gc
from sklearn.preprocessing import StandardScaler

In [ ]:
# Importando os dados dos arquivos, já eliminando cabeçalho e rodapé na abertura
atu = pd.read_excel("ATU_ESCOLAS_2021.xlsx", skiprows=8, nrows=174179)
dsu = pd.read_excel("DSU_ESCOLAS_2021.xlsx", skiprows=9, nrows=178126)
had = pd.read_excel("HAD_ESCOLAS_2021.xlsx", skiprows=8, nrows=151777)
icg = pd.read_excel("ICG_ESCOLAS_2021.xlsx", skiprows=10, nrows=178370)
ird = pd.read_excel("IRD_ESCOLAS_2021.xlsx", skiprows=10, nrows=165853)
tdi = pd.read_excel("TDI_ESCOLAS_2021.xlsx", skiprows=8, nrows=130112)
txr = pd.read_excel("TX_REND_ESCOLAS_2021.xlsx", skiprows=8, nrows=130129)

In [ ]:
# Utilizado para remover os warnings 
import warnings
warnings.filterwarnings('ignore')
# Usado para plotar na célula ao invés de abrir uma nova janela
%matplotlib inline

In [ ]:
# Função para mostrar a estrutura de colunas do dataframe
# dataframes = Lista com dataframes
# cabeçalhos = Lista com cabeçalhos
# nlinhas = Quantidade máxima de linhas
def mostrar_estrutura(dataframes, cabecalhos, nlinhas):
    tabelas = []
    for df, cabecalho in zip(dataframes, cabecalhos):
        colunas = df.columns.tolist()
        tabela = PrettyTable()
        tabela.field_names = [f"DataFrame {cabecalho}"]

        # Adicione as colunas até o máximo de linhas
        while len(colunas) < nlinhas:
            colunas.append("")  # Adicione uma linha vazia

        tabela.add_row(["\n".join(colunas)])
        tabelas.append(tabela)

    # Combina as linhas das tabelas
    linha_combinada = list(zip_longest(*[str(tabela).splitlines() for tabela in tabelas], fillvalue=''))

    # Imprime as tabelas lado a lado
    for linha in linha_combinada:
        print(' '.join(linha))

In [ ]:
# Chamando a funcão para mostrar a estrutura de 5 arquivos
mostrar_estrutura([txr, atu, had, tdi, dsu], ["TXR", "ATU", "HAD", "TDI", "DSU"], 10)

In [ ]:
#Deixando apenas as variáveis necessárias
atu = atu.iloc[:, [1,2,4,5,6,7,8,12,25]]
dsu = dsu.iloc[:, [1,2,4,5,6,7,8,12,15]]
had = had.iloc[:, [1,2,4,5,6,7,8,12,24]]
icg = icg.iloc[:, [1,2,4,5,6,7,8,9]]
ird = ird.iloc[:, [1,2,4,5,6,7,8,9]]
tdi = tdi.iloc[:, [1,2,4,5,6,7,8,9,21]]
txr = txr.iloc[:, [1,2,4,5,6,7,8,9,21]]

In [ ]:
# Renomear variaveis
# Foi criada uma lista com o nome das variáveis da chave composta
colunas = ['regiao', 'uf', 'municipio', 'id_escola', 'escola', 'categoria', 'dependencia']

atu.columns = colunas + ['vr_fun_atu', 'vr_med_atu']
dsu.columns = colunas + ['vr_fun_dsu', 'vr_med_dsu']
had.columns = colunas + ['vr_fun_had', 'vr_med_had']
icg.columns = colunas + ['nivel']
ird.columns = colunas + ['vr_ird']
tdi.columns = colunas + ['vr_fun_tdi', 'vr_med_tdi']
txr.columns = colunas + ['vr_fun_txr', 'vr_med_txr']

In [ ]:
# Chamando a funcão para mostrar a estrutura de todos os arquivos após os ajustes
mostrar_estrutura([txr, atu, had, tdi, dsu], ["TXR", "ATU", "HAD", "TDI", "DSU"], 9)

In [ ]:
# Unindo os dataframes em um único (juntando tudo)
# Realizar o join dos DataFrames (usamos a lista com as colunas da chave composta)
colunas_para_merge = colunas
dataframes = [txr, atu, had, tdi, dsu, icg, ird]
# Começar com o primeiro DataFrame
df_inep = dataframes[0]
# Realizar o join dos DataFrames usando as colunas em comum como chave
for df in dataframes[1:]:
    df_inep = pd.merge(df_inep, df, on=colunas_para_merge, how='inner')

In [ ]:
# Verificar a estrutura do df_inep
mostrar_estrutura([df_inep], ["DF_INEP"], 10)

In [ ]:
# Verificando o total de registros antes da preparação dos dados
qt_antes = len(df_inep)
print(f"Qt antes: {qt_antes} registros")

In [ ]:
# Remover registros que contenham "--" e NaN
#Substituir '--' por NaN para facilitar a remoção
df_inep = df_inep.replace('--', pd.NA)
# Remover as linhas que contêm pelo menos um NaN (anteriormente '--')
df_inep = df_inep.dropna()

In [ ]:
# Remover possíveis linhas duplicadas,mantendo a primeira ocorrência
df_inep = df_inep.drop_duplicates(subset= colunas, keep='first')

In [ ]:
# Verificando o total de registros após a preparação dos dados
qt_apos = len(df_inep)
qt_removido = qt_antes - qt_apos
qt_duplicados = qt_apos - df_inep['id_escola'].nunique()
print(f"Qt. antes: {qt_antes}, Qt após: {qt_apos}, Qt removidos: {qt_removido}, Qt duplicados: {qt_duplicados}")

In [ ]:
#Listando os types das colunas
df_inep.infer_objects()
df_inep.dtypes

In [ ]:
# Defina as colunas de texto como tipo string, usaremos a lista já existente em colunas
colunas_texto = colunas + ['nivel']
df_inep[colunas_texto] = df_inep[colunas_texto].astype(str)
# Defina as demais colunas numéricas para terem duas casas decimais
colunas_numericas = [coluna for coluna in df_inep.columns if coluna not in colunas_texto]
df_inep[colunas_numericas] = df_inep[colunas_numericas].round(2).astype(float)

In [ ]:
colunas_numericas

In [ ]:
#Listando os types das colunas
df_inep.infer_objects()
df_inep.dtypes

In [ ]:
# Analisando a variavel nivel para aplicar dummyzação
df_inep[['municipio', 'vr_ird', 'nivel']].head(10)

In [ ]:
# Dummyzando a variável nivel
df_inep = pd.get_dummies(df_inep, columns=['nivel'], prefix=['d'])
# Renomeando as novas colunas
df_inep.rename(columns={
    'd_Nível 2': 'nivel2', 
    'd_Nível 3': 'nivel3', 
    'd_Nível 4': 'nivel4',
    'd_Nível 5': 'nivel5',
    'd_Nível 6': 'nivel6'
}, inplace=True)

In [ ]:
# Analisando a dummyzação
colunas_dummies = ['nivel2', 'nivel3', 'nivel4', 'nivel5', 'nivel6']
df_inep[['municipio', 'vr_ird'] + colunas_dummies].head(10)

In [ ]:
# Verificando se tem registro duplicado para a coluna id_cidade
print(f"Qtde registros: {len(df_inep)} - Qtde duplicados: {len(df_inep) - df_inep['id_escola'].nunique()}")

In [ ]:
#Analisando antes de padronizar
df_inep[['id_escola', 'vr_fun_txr', 'vr_med_txr', 'vr_fun_atu', 'vr_med_atu', 'nivel2']]

In [ ]:
# Padronização das variáveis
# Atenção, pacote: from sklearn.preprocessing import StandardScaler
# Inicialize o objeto StandardScaler
scaler = StandardScaler()
# Ajuste e transforme o DataFrame (aqui usamos as duas listas que contém o que precisamos)
colunas_a_padronizar = colunas_numericas + colunas_dummies
# Padronize as colunas selecionadas
df_inep[colunas_a_padronizar] = scaler.fit_transform(df_inep[colunas_a_padronizar])

In [ ]:
# Analisando após padronizar 
df_inep[['id_escola', 'vr_fun_txr', 'vr_med_txr', 'vr_fun_atu', 'vr_med_atu', 'nivel2']]

In [ ]:
# Criar os datraframes para aplicar a PCA para os ensinos fundamental e médio
df_fundamental = df_inep[['id_escola', 'vr_fun_atu','vr_fun_dsu','vr_fun_had','vr_ird','vr_fun_tdi','vr_fun_txr'] + colunas_dummies]
df_medio = df_inep[['id_escola', 'vr_med_atu','vr_med_dsu','vr_med_had','vr_ird','vr_med_tdi','vr_med_txr'] + colunas_dummies]

In [ ]:
colunas_dummies

In [ ]:
mostrar_estrutura([df_inep, df_fundamental, df_medio], ["DF_INEP", "DF_FUNDAMENTAL", "DF_MEDIO"], 10)

In [ ]:
df_fundamental

In [ ]:
id_escola_f = df_fundamental['id_escola'].values
id_escola_m = df_medio['id_escola'].values
# Removendo a coluna
df_fundamental.drop('id_escola', axis=1, inplace=True)
df_medio.drop('id_escola', axis=1, inplace=True)
# Nomeando os registros com o id_escola
df_fundamental.index = id_escola_f
df_medio.index = id_escola_m

In [ ]:
df_fundamental

In [ ]:
df_medio

In [ ]:
# Removendo o indicador de desempenho de complexidade da gestão escola
df_fundamental = df_fundamental.drop(columns=colunas_dummies)
df_medio = df_medio.drop(columns=colunas_dummies)
df_inep = df_inep.drop(columns=colunas_dummies)

In [ ]:
mostrar_estrutura([df_inep, df_fundamental, df_medio], ["DF_INEP", "DF_FUNDAMENTAL", "DF_MEDIO",], 5)

In [ ]:
#Explorando como ficaram os dataframes para o ensino fundamental e médio
dataframes = [df_fundamental, df_medio]
for idx, df in enumerate(dataframes, start=1):
    print(f"DataFrame {idx}:\n")
    print(df.head())
    print("\n" + "="*30 + "\n")

In [ ]:
# Deletando os dataframes auxiliares
del atu
del dsu
del had
del icg
del ird
del tdi
del txr
# Forçar a coleta de lixo para liberar a memória
gc.collect()